In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# FRAMEWORM Quick Start\n",
    "\n",
    "Train your first model in 5 minutes.\n",
    "\n",
    "[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/frameworm/blob/main/examples/notebooks/01_quickstart.ipynb)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install FRAMEWORM\n",
    "!pip install frameworm -q"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Import & Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "from torch.utils.data import DataLoader, TensorDataset\n",
    "from frameworm import Config, get_model, Trainer\n",
    "\n",
    "# Check GPU\n",
    "device = 'cuda' if torch.cuda.is_available() else 'cpu'\n",
    "print(f'Using device: {device}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Create Dummy Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create synthetic dataset\n",
    "X_train = torch.randn(1000, 3, 32, 32)\n",
    "X_val = torch.randn(200, 3, 32, 32)\n",
    "\n",
    "train_loader = DataLoader(TensorDataset(X_train), batch_size=64, shuffle=True)\n",
    "val_loader = DataLoader(TensorDataset(X_val), batch_size=64)\n",
    "\n",
    "print(f'Train batches: {len(train_loader)}')\n",
    "print(f'Val batches: {len(val_loader)}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Configure Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "config = Config.from_dict({\n",
    "    'model': {\n",
    "        'type': 'vae',\n",
    "        'in_channels': 3,\n",
    "        'latent_dim': 64,\n",
    "        'hidden_dim': 128\n",
    "    }\n",
    "})\n",
    "\n",
    "model = get_model('vae')(config)\n",
    "print(f'Model parameters: {sum(p.numel() for p in model.parameters()):,}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Train"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "optimizer = torch.optim.Adam(model.parameters(), lr=0.001)\n",
    "trainer = Trainer(model, optimizer, device=device)\n",
    "\n",
    "# Train for 10 epochs\n",
    "trainer.train(train_loader, val_loader, epochs=10)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Generate Samples"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import matplotlib.pyplot as plt\n",
    "\n",
    "model.eval()\n",
    "with torch.no_grad():\n",
    "    z = torch.randn(16, config.model.latent_dim).to(device)\n",
    "    samples = model.decode(z)\n",
    "\n",
    "fig, axes = plt.subplots(4, 4, figsize=(10, 10))\n",
    "for i, ax in enumerate(axes.flat):\n",
    "    img = samples[i].cpu().permute(1, 2, 0)\n",
    "    img = (img + 1) / 2  # Denormalize\n",
    "    ax.imshow(img.clamp(0, 1))\n",
    "    ax.axis('off')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Next Steps\n",
    "\n",
    "- Try other models: `dcgan`, `ddpm`, `vqvae2`\n",
    "- Add callbacks for early stopping\n",
    "- Track experiments with W&B\n",
    "- Export and deploy your model\n",
    "\n",
    "Check out more tutorials at [frameworm.readthedocs.io](https://frameworm.readthedocs.io)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}